In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Embedding, Dense

In [2]:
# Texto base
frase = "o gato está em cima"

In [3]:
# Tokenização
tokenizer = Tokenizer()
tokenizer.fit_on_texts([frase])
sequences = tokenizer.texts_to_sequences([frase])[0]
vocab_size = len(tokenizer.word_index) + 1  # +1 para o índice 0

In [4]:
# Criar pares (entrada -> próxima palavra)
X = []
y = []
for i in range(1, len(sequences)):
    X.append(sequences[:i])
    y.append(sequences[i])

In [5]:
# Padronizar tamanhos das sequências (preenchendo com 0 à esquerda)
from tensorflow.keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X, maxlen=max([len(seq) for seq in X]), padding='pre')
y = to_categorical(y, num_classes=vocab_size)

In [6]:
# Criar o modelo
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=X.shape[1]))
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [7]:
# Treinar
model.fit(X, y, epochs=300, verbose=0)

In [8]:
# Função para prever a próxima palavra
def prever_proxima(palavras):
    seq = tokenizer.texts_to_sequences([palavras])[0]
    seq = pad_sequences([seq], maxlen=X.shape[1], padding='pre')
    pred = model.predict(seq, verbose=0)
    idx = np.argmax(pred)
    for palavra, i in tokenizer.word_index.items():
        if i == idx:
            return palavra
    return "?"

In [9]:
# Testar
entrada = "o gato está"
proxima = prever_proxima(entrada)
print(f"Próxima palavra depois de '{entrada}': '{proxima}'")

Próxima palavra depois de 'o gato está': 'em'
